In [ ]:
!pip install mapie

In [ ]:
# load packages
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_percentage_error as mape, mean_squared_error as mse
from pyarrow import feather as pq
import geopandas as gpd
import folium
from folium import Marker
from shapely import geometry
from tqdm import tqdm
pd.set_option('display.max_columns', None)
from ipywidgets import interact
import scipy
from tqdm import tqdm
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
import mapie
from mapie import regression
from mapie.metrics import regression_coverage_score
from mapie.regression import MapieRegressor
from mapie.quantile_regression import MapieQuantileRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from mapie.subsample import BlockBootstrap
from mapie.time_series_regression import MapieTimeSeriesRegressor
#import shap
# Feature Importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_pinball_loss
from sklearn.metrics import make_scorer
#conda install tensorflow
#from models import regression_model
#import data_preprocessing
#from conformal_prediction import EnCQR
#import utils
#import data_loaders

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
# load data for monthly performance
data = pd.read_csv("/content/gdrive/MyDrive/Aurora_Thesis/data_converted.csv")

Mounted at /content/gdrive


In [ ]:
# load data
#data = pd.read_csv("data_converted.csv")
data.time = pd.to_datetime(data.time)
data = data.reset_index()
# need to manually add 2 station Id
data.loc[data.station== "Bologna (BO)", "station_id"] = "ID1999"
data.loc[data.station== "San Pietro Capofiume (SPC)", "station_id"] = "ID1998"
# switch London data
data["OAtot_2"] = data.HOA_PMF + data.BBOA_PMF + data.OOAtot_PMF
data.loc[data.station == "London","OAtot_PMF"] = data.loc[data.station == "London","OAtot_2"]
# Remove Zurich 2017
data = data.loc[(data.station != "Zurich") | (data.year != 2017),:]

In [ ]:
# CLEAN DATA
# remove OA with less than 0.1
data= data.loc[data.OAtot_PMF >= 0.1, :]
# and stations with less than 30 obs.
select = (data.groupby("station_id")["OAtot_PMF"].size() > 30).reset_index()
data = data.set_index("station_id")
data = data.join(select.set_index("station_id"), rsuffix = "keep")
data = data.loc[ data.OAtot_PMFkeep == True, :]
data = data.reset_index()
# add day of week
data["day_week"] = data.time.dt.day_of_week

In [ ]:
# feature engineering
data["rc_1_1000-rc_1_100"] = data["road_class_1_1000"] - data["road_class_1_100"]
data["rc_2_1000-rc_2_100"] = data["road_class_2_1000"] - data["road_class_2_100"]
data["rc_3_1000-rc_3_100"] = data["road_class_3_1000"] - data["road_class_3_100"]
# CAMX proportions of components of OA
data["p_HOA"] = data["HOA_CAMX"] / data["OAtot_CAMX"]
data["p_BBOA"] = data["BBOA_CAMX"] / data["OAtot_CAMX"]
data["p_OOAtot"] = data["OOAtot_CAMX"] / data["OAtot_CAMX"]

# need to decorralate some land-use variables
data["diff_agriculture"] = data["agriculture1000"] - data["agriculture500"]
data["diff_airports"] = data["airports1000"] - data["airports500"]
data["diff_barren"] = data["barren1000"] - data["barren500"]
data["diff_industrial"] = data["industrial1000"] - data["industrial500"]
data["diff_industrial_transport"]= data["industrial_transport1000"] - data["industrial_transport500"]
data["diff_natural_green"] =  data["natural_green1000"] - data["natural_green500"]
data["diff_ports"] = data["ports1000"] - data["ports500"]
data["diff_roads_rails"] = data["roads_rails1000"] - data["roads_rails500"]
data["diff_snow_ice"] = data["snow_ice1000"] - data["snow_ice500"]
data["diff_transport"] = data["transport1000"] - data["transport500"]
data["diff_urban_fabric"] = data["urban_fabric1000"] - data["urban_fabric500"]
data["diff_urban_green"] = data["urban_green1000"] - data["urban_green500"]
data["diff_water"] = data["water1000"] - data["water500"]
data["diff_wetlands"] = data["wetlands1000"] - data["wetlands500"]
# also for Population and IMD
data["diff_population"] = data["population_1000"] - data["population_500"]
data["diff_imd"] = data["imd1000"] - data["imd500"]

In [ ]:
# eventually need this
# stations with at least 100 observations
keep = (data.groupby("station_id")["OAtot_PMF"].size() > 100).reset_index()
# only stations with at least 100 observations
select_data = data.copy(deep=True)

for stat_id in data.station_id.unique():
    if keep.loc[keep.station_id == stat_id, "OAtot_PMF"].iloc[0] == False:
        select_data = data.loc[data.station_id != stat_id,:]

print("data",data.shape )
print("select", select_data.shape)

data (26049, 150)
select (25967, 150)


In [ ]:
# Get OA
data = data.loc[data.OAtot_PMF.isnull()==False,:]
data = data.sort_values(by="time")
# Get Y
Y = data.loc[:, ["time","station_id","station","OAtot_PMF"]]
Y = Y.set_index("time")
# Get X design
# probabaly better the 3 components
covars = ["time", "station", "station_id", "OAtot_PMF", "HOA_CAMX", "BBOA_CAMX", "OOAtot_CAMX", "year", "month","day_week",
"temp_CAMX", "rh_CAMX", "press_CAMX", "ws_CAMX", "wd_CAMX", "pblh_CAMX", "wind_x_CAMX", "wind_y_CAMX","diff_agriculture", "diff_airports", "diff_barren", "diff_industrial", "diff_industrial_transport",
    "diff_natural_green", "diff_ports", "diff_roads_rails", "diff_snow_ice", "diff_transport", "diff_urban_fabric", "diff_urban_green", "diff_water", "diff_wetlands",
    "agriculture500","airports500", "barren500", "industrial500", "industrial_transport500", "natural_green500", "ports500", "roads_rails500", "snow_ice500",
    "transport500", "urban_fabric500","urban_green500", "water500", "wetlands500","diff_imd","imd500", "diff_population", "population_500","elevation","Lat","Lon", "area_grid",
    "distance_border", "distance_mt"]
X = data.loc[: , covars]
X = X.set_index("time")
X = X.fillna(0)

In [ ]:
# get train/cal/test df
# generate train dataset and test dataset
X_train = pd.DataFrame(np.zeros(X.shape[1])).T
X_cal = pd.DataFrame(np.zeros(X.shape[1])).T
X_test = pd.DataFrame(np.zeros(X.shape[1])).T
Y_train = pd.Series(0)
Y_cal = pd.Series(0)
Y_test = pd.Series(0)

for station_id in tqdm(data.station_id.unique()):
    # get station
    X_station = X.loc[X.station_id == station_id, :]
    Y_station = Y.loc[Y.station_id == station_id, :]
    # train and test split (as in training data proportion)
    split = 0.75
    index_train = int(np.floor(len(X_station) * split))
    Y_train_station = Y_station.iloc[:index_train,:]
    Y_test_station = Y_station.iloc[index_train:, :]
    X_train_station = X_station.iloc[:index_train,:]
    X_test_station = X_station.iloc[index_train:,:]

    # additionally split train in half to have a calibration dataset
    split = 0.5
    index_train = int(np.floor(len(X_train_station) * split))
    Y_train_stat = Y_train_station.iloc[:index_train,:]
    Y_cal_station = Y_train_station.iloc[index_train:, :]
    X_train_stat = X_train_station.iloc[:index_train,:]
    X_cal_station = X_train_station.iloc[index_train:,:]

    # store in df
    X_train = pd.concat((X_train, X_train_stat),axis = 0)
    X_cal = pd.concat((X_cal, X_cal_station),axis = 0)
    X_test = pd.concat((X_test, X_test_station),axis = 0)
    Y_train = pd.concat ((Y_train, Y_train_stat),axis = 0)
    Y_cal = pd.concat ((Y_cal, Y_cal_station),axis = 0)
    Y_test = pd.concat ((Y_test, Y_test_station),axis = 0)

# remove first row/obs. of each df (fix here maybe)
X_train = X_train.iloc[1:, 55:]
X_train = X_train.fillna(0)
X_cal = X_cal.iloc[1:, 55:]
X_cal = X_cal.fillna(0)
X_test = X_test.iloc[1:, 55:]
X_test = X_test.fillna(0)
Y_train = Y_train.iloc[1:]
Y_cal = Y_cal.iloc[1:]
Y_test = Y_test.iloc[1:]

100%|██████████| 102/102 [00:11<00:00,  8.60it/s]


In [ ]:
# helper functions to compute weighted quantiles
def weighted_quantile(values, sample_weight, quantiles,
                      values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of
        initial array
    :param old_style: if True, will correct output to be consistent
        with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

In [ ]:
# define weighted Conformal Prediction Sets
def weighted_conformal(model, decay_rate, alpha, X_train, Y_train, X_cal, Y_cal, X_test, Y_test):
    #model.fit(X_train, Y_train)
    y_pred_cal = model.predict(X_cal)
    conformity_scores = np.abs(Y_cal - y_pred_cal)
    n_cal = len(Y_cal)
    t = np.array([n_cal-i for i in range(n_cal)])
    # define the fixed weights
    weights = decay_rate**((n_cal + 1)-t)
    # define the normalized weights
    normal_weights = weights/(weights.sum() + 1)
    # get the (1-alpha) weighted quantile
    q_hat = weighted_quantile(conformity_scores, normal_weights, ((1-alpha) * (1 + 1/n_cal)) )
    #plt.figure(figsize=(10,5))
    #plt.hist(conformity_scores)
    #print(q_hat)
    y_pred_test = model.predict(X_test)
    lower_bound = y_pred_test - q_hat
    upper_bound = y_pred_test + q_hat
    return y_pred_test, lower_bound, upper_bound

In [ ]:
def weighted_QCR(model_lower, model_upper, decay_rate, alpha, X_train, Y_train, X_cal, Y_cal, X_test, Y_test):
    #model_lower.fit(X_train, Y_train)
    #model_upper.fit(X_train, Y_train)
    # prediction on calibration set
    y_pred_lo = model_lower.predict(X_cal)
    y_pred_up = model_upper.predict(X_cal)
    # obtain conformity score
    conformity_scores = np.zeros(len(Y_cal))
    for i in range(len(conformity_scores)):
        conformity_scores[i] = np.max((Y_cal[i] - y_pred_up[i], y_pred_lo[i] - Y_cal[i]))

    n_cal = len(Y_cal)
    t = np.array([n_cal-i for i in range(n_cal)])
    # define the fixed weights
    weights = decay_rate**((n_cal + 1)-t)
    # define the normalized weights
    normal_weights = weights/(weights.sum() + 1)
    # get the (1-alpha) weighted quantile
    q_hat = weighted_quantile(conformity_scores, normal_weights,  ((1-alpha) * (1 + 1/n_cal)))
    #plt.figure(figsize=(10,5))
    #plt.hist(conformity_scores)
    #print(q_hat)
    lower_bound = model_lower.predict(X_test) - q_hat
    upper_bound = model_upper.predict(X_test) - q_hat
    return  lower_bound, upper_bound

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# train Boosting on train data, both for mean and for quantiles
covars = ["HOA_CAMX", "BBOA_CAMX", "OOAtot_CAMX", "year", "month","day_week",
"temp_CAMX", "rh_CAMX", "press_CAMX", "ws_CAMX", "wd_CAMX", "pblh_CAMX", "wind_x_CAMX", "wind_y_CAMX","diff_agriculture", "diff_airports", "diff_barren", "diff_industrial", "diff_industrial_transport",
    "diff_natural_green", "diff_ports", "diff_roads_rails", "diff_snow_ice", "diff_transport", "diff_urban_fabric", "diff_urban_green", "diff_water", "diff_wetlands",
    "agriculture500","airports500", "barren500", "industrial500", "industrial_transport500", "natural_green500", "ports500", "roads_rails500", "snow_ice500",
    "transport500", "urban_fabric500","urban_green500", "water500", "wetlands500","diff_imd","imd500", "diff_population", "population_500","elevation","Lat","Lon", "area_grid",
    "distance_border", "distance_mt"]
alpha = 0.1
rf = RandomForestRegressor(random_state=5, n_estimators=500)
boost = GradientBoostingRegressor(max_depth = 3, learning_rate= 0.1, random_state=5)
boost_median = GradientBoostingRegressor(max_depth = 3, learning_rate= 0.1, random_state=5, loss = "quantile", alpha = 0.5)
# tune quantiles
qlo = GradientBoostingRegressor(loss = "quantile", alpha = alpha/2, max_depth = 3, learning_rate= 0.1, random_state=5)
qup = GradientBoostingRegressor(loss = "quantile", alpha = 1-alpha/2, max_depth = 3, learning_rate= 0.1, random_state=5)
mean_pinball_loss_lo = make_scorer(mean_pinball_loss, alpha=alpha/2, greater_is_better=False)
mean_pinball_loss_up = make_scorer(mean_pinball_loss, alpha=1- alpha/2, greater_is_better=False)
boost_param =  {"max_depth":[3,5,7], "learning_rate":[0.1,0.01,0.05]}
clf = RandomizedSearchCV(qlo, boost_param,n_jobs=-1, scoring = mean_pinball_loss_lo, random_state=50)
clf.fit(X_train[covars], Y_train.OAtot_PMF)
clf2 = RandomizedSearchCV(qup, boost_param, n_jobs=-1, scoring = mean_pinball_loss_up, random_state=50)
clf2.fit(X_train[covars], Y_train.OAtot_PMF)
rf.fit(X_train[covars], Y_train.OAtot_PMF)
boost.fit(X_train[covars], Y_train.OAtot_PMF)
boost_median.fit(X_train[covars], Y_train.OAtot_PMF)
#qlo.fit(X_train[covars], Y_train.OAtot_PMF)
#qup.fit(X_train[covars], Y_train.OAtot_PMF)

GradientBoostingRegressor(alpha=0.5, loss='quantile', random_state=5)

In [ ]:
# train EnbPI
n_splits = 10 # how to choose?
tscv = TimeSeriesSplit(n_splits = n_splits)
rf = RandomForestRegressor(random_state=10)
#rf_param = {"max_features":["sqrt","log2"]}
#clf_forest = GridSearchCV(rf, rf_param, cv = tscv, n_jobs=-1)
#clf_forest.fit(X_train[covars], Y_train["OAtot_PMF"])
# for ts sampling of data
# simple RF
rf = RandomForestRegressor(random_state=10)
# n_resamplings = B, length = s, agg_function = maybe better quantile?
cv_mapiets = BlockBootstrap(n_resamplings=50, length= int(len(Y_train)/2), overlapping=True, random_state=59)
# fit EnbPI
mapie_enbpi = MapieTimeSeriesRegressor(rf, method = "enbpi",
                                            cv = cv_mapiets, agg_function = "mean", n_jobs = -1)
# mapie_enbpi = mapie_enbpi.fit(X_train[covars], Y_train["OAtot_PMF"])

In [ ]:
# conformalize each station
# NAIVE/CQR/QUANTILE/WeightNAIVE/WeightCQR/EnbPI
def result_conformal(stat_id):
    rho = 0.99
    # get station data
    X_train_stat = X_train.loc[X_train.station_id == stat_id,:]
    Y_train_stat = Y_train.loc[X_train.station_id == stat_id, "OAtot_PMF"]
    X_cal_stat = X_cal.loc[X_cal.station_id == stat_id, covars]
    X_test_stat = X_test.loc[X_test.station_id == stat_id, covars]
    Y_cal_stat = Y_cal.loc[Y_cal.station_id == stat_id , "OAtot_PMF"]
    Y_test_stat = Y_test.loc[Y_test.station_id == stat_id , "OAtot_PMF"]

    # NAIVE
    mapie = MapieRegressor(boost, cv = "prefit", method = "base")
    mapie.fit(X_cal_stat, Y_cal_stat)
    y_pred_naive, y_pis_naive = mapie.predict(X_test_stat, alpha = alpha)
    coverage_naive = regression_coverage_score(Y_test_stat, y_pis_naive[:, 0, 0], y_pis_naive[:, 1, 0])
    wid_naive = (y_pis_naive[:, 1, 0] - y_pis_naive[:, 0, 0]).mean()
    nor_wid_naive = wid_naive/(Y_test_stat.max() - Y_test_stat.min())

    # CQR
    mapie = MapieQuantileRegressor(boost_median, cv = "split", method = "quantile", alpha = alpha)
    mapie.fit(X_train[covars], Y_train.OAtot_PMF, X_calib = X_cal_stat, y_calib = Y_cal_stat)
    y_pred_cqr, y_pis_cqr = mapie.predict(X_test_stat, alpha = alpha)
    # try my own implementation
    coverage_cqr = regression_coverage_score(Y_test_stat, y_pis_cqr[:, 0, 0], y_pis_cqr[:, 1, 0])
    wid_cqr = (y_pis_cqr[:, 1, 0] - y_pis_cqr[:, 0, 0]).mean()
    nor_wid_cqr= wid_cqr/(Y_test_stat.max() - Y_test_stat.min())

    # QUANTILE
    y_pred_quantile = boost_median.predict(X_test_stat)
    y_pis_quantile_l = clf.predict(X_test_stat)
    y_pis_quantile_u = clf2.predict(X_test_stat)
    coverage_quantile = regression_coverage_score(Y_test_stat, y_pis_quantile_l, y_pis_quantile_u)
    wid_quantile = (y_pis_quantile_u - y_pis_quantile_l).mean()
    nor_wid_quantile= wid_quantile/(Y_test_stat.max() - Y_test_stat.min())

    # weighted NAIVE
    y_pred_weighted, y_pis_lower, y_pis_upper = weighted_conformal(boost,  rho, alpha, X_train_stat, Y_train_stat, X_cal_stat, Y_cal_stat, X_test_stat, Y_test_stat)
    coverage_weighted = regression_coverage_score(Y_test_stat, y_pis_lower, y_pis_upper)
    wid_weighted = (y_pis_upper - y_pis_lower).mean()
    nor_wid_weight= wid_weighted/(Y_test_stat.max() - Y_test_stat.min())

    # Weighted CQR
    y_pis_lowerCQR, y_pis_upperCQR = weighted_QCR(clf, clf2, rho , alpha, X_train_stat, Y_train_stat, X_cal_stat, Y_cal_stat, X_test_stat, Y_test_stat)
    coverage_weightedCQR = regression_coverage_score(Y_test_stat, y_pis_lowerCQR, y_pis_upperCQR)
    wid_weightedCQR = (y_pis_upperCQR - y_pis_lowerCQR).mean()
    nor_wid_weightCQR = wid_weightedCQR/(Y_test_stat.max() - Y_test_stat.min())

    # EnbPI
    rf = RandomForestRegressor(random_state=10)
    # n_resamplings = B, length = s, agg_function = maybe better quantile?
    cv_mapiets = BlockBootstrap(n_resamplings=50, length= int(len(Y_train_stat)/2), overlapping=True, random_state=59)
    # fit EnbPI
    mapie_enbpi = MapieTimeSeriesRegressor(rf, method = "enbpi",
                                            cv = cv_mapiets, agg_function = "mean", n_jobs = -1)
    mapie_enbpi = mapie_enbpi.fit(pd.concat((X_train_stat[covars], X_cal_stat[covars]),axis = 0), pd.concat((Y_train_stat, Y_cal_stat),axis = 0))
    y_pred_enbpi, y_pis_enbpi = mapie_enbpi.predict(X_test_stat, alpha = alpha, ensemble = True, optimize_beta = False) # # maybe betteer false #true or false
    coverage_enbpi = regression_coverage_score(Y_test_stat, y_pis_enbpi[:, 0, 0], y_pis_enbpi[:, 1, 0])
    wid_enbpi = (y_pis_enbpi[:, 1, 0] - y_pis_enbpi[:, 0, 0]).mean()
    nor_wid_enbpi= wid_enbpi/(Y_test_stat.max() - Y_test_stat.min())

    n_test = len(Y_test_stat)


    return coverage_naive, nor_wid_naive, coverage_cqr, nor_wid_cqr, coverage_quantile, nor_wid_quantile, coverage_weighted, nor_wid_weight,coverage_weightedCQR, nor_wid_weightCQR, coverage_enbpi, nor_wid_enbpi, n_test

changing B and S lead to an improvement!
try not to optimize beta, did not change anything!
then to aggregate differentely


 then no ensemble, then read about B and s

In [ ]:
#get results (unweighted)

# dict to save results
w_naive = {}
cov_naive = {}
w_cqr = {}
cov_cqr = {}
w_qr = {}
cov_qr = {}
w_nw = {}
cov_nw = {}
w_cqrw = {}
cov_cqrw = {}
w_enbpi = {}
cov_enbpi = {}
for stat in tqdm(data.station_id.unique()):
  # re-arrange order
  cov_naive[stat],w_naive[stat], cov_cqr[stat],w_cqr[stat], cov_qr[stat],w_qr[stat], cov_nw[stat],w_nw[stat], cov_cqrw[stat],w_cqrw[stat], cov_enbpi[stat], w_enbpi[stat], n = result_conformal(stat)


100%|██████████| 102/102 [1:05:14<00:00, 38.38s/it]


In [ ]:
# unweighted mean
print("avg. width naive: ", np.mean(list(w_naive.values())))
print("avg. cov naive: ", np.mean(list(cov_naive.values())))
print("avg. width cqr: ", np.mean(list(w_cqr.values())))
print("avg. cov cqr: ", np.mean(list(cov_cqr.values())))
print("avg. width qr: ", np.mean(list(w_qr.values())))
print("avg. cov qr: ", np.mean(list(cov_qr.values())))
print("avg. width weigth naive: ", np.mean(list(w_nw.values())))
print("avg. cov weigth naive: ", np.mean(list(cov_nw.values())))
print("avg. width weight cqr: ", np.mean(list(w_cqrw.values())))
print("avg. cov weight cqr: ", np.mean(list(cov_cqrw.values())))
print("avg. width enbpi: ", np.mean(list(w_enbpi.values())))
print("avg. cov enbpi: ", np.mean(list(cov_enbpi.values())))

avg. width naive:  0.9417962278095121
avg. cov naive:  0.827748681995528
avg. width cqr:  1.068373279609097
avg. cov cqr:  0.863272712269827
avg. width qr:  1.1002242506033844
avg. cov qr:  0.868193592057597
avg. width weigth naive:  1.0453366621668074
avg. cov weigth naive:  0.857758187833763
avg. width weight cqr:  1.1002242506033844
avg. cov weight cqr:  0.8094700564257874
avg. width enbpi:  0.491624416310531
avg. cov enbpi:  0.6356418413017507


In [ ]:
# unweighted std dev
print("std. width naive: ", np.std(list(w_naive.values())))
print("std. cov naive: ", np.std(list(cov_naive.values())))
print("std. wid cqr: ", np.std(list(w_cqr.values())))
print("std. cov cqr: ", np.std(list(cov_cqr.values())))
print("std. wid qr: ", np.std(list(w_qr.values())))
print("std. cov qr: ", np.std(list(cov_qr.values())))
print("std. wid weigth naive: ", np.std(list(w_nw.values())))
print("std. cov weigth naive: ", np.std(list(cov_nw.values())))
print("std. wid weight cqr: ", np.std(list(w_cqrw.values())))
print("std. cov weight cqr: ", np.std(list(cov_cqrw.values())))
print("std. wid enbpi: ", np.std(list(w_enbpi.values())))
print("std. cov enbpi: ", np.std(list(cov_enbpi.values())))

std. width naive:  0.9700161706188613
std. cov naive:  0.18550229236284355
std. wid cqr:  0.935714323580057
std. cov cqr:  0.14344426726205436
std. wid qr:  0.9709315156281493
std. cov qr:  0.13914491742795934
std. wid weigth naive:  1.0513609796551666
std. cov weigth naive:  0.1761363894398994
std. wid weight cqr:  0.9709315156281492
std. cov weight cqr:  0.21489074754858223
std. wid enbpi:  0.5942566312846672
std. cov enbpi:  0.19474393774147283


In [ ]:
# median
print("median width naive: ", np.median(list(w_naive.values())))
print("median. cov naive: ", np.median(list(cov_naive.values())))
print("median. width cqr: ", np.median(list(w_cqr.values())))
print("median. cov cqr: ", np.median(list(cov_cqr.values())))
print("median. width qr: ", np.median(list(w_qr.values())))
print("median. cov qr: ", np.median(list(cov_qr.values())))
print("median. width weigth naive: ", np.median(list(w_nw.values())))
print("median. cov weigth naive: ", np.median(list(cov_nw.values())))
print("median. width weight cqr: ", np.median(list(w_cqrw.values())))
print("median. cov weight cqr: ", np.median(list(cov_cqrw.values())))
print("median. width enbpi: ", np.median(list(w_enbpi.values())))
print("median. cov enbpi: ", np.median(list(cov_enbpi.values())))

median width naive:  0.9072481572481572
median. cov naive:  0.7283594764207223
median. width cqr:  0.9047619047619048
median. cov cqr:  0.733069896246856
median. width qr:  0.9
median. cov qr:  0.6546477470625605
median. width weigth naive:  0.9346377306903623
median. cov weigth naive:  0.7820729520975818
median. width weight cqr:  0.8738636363636363
median. cov weight cqr:  0.7550779091395974
median. width enbpi:  0.6830429732868757
median. cov enbpi:  0.3569237043297161


In [ ]:
# GET RESULTS for weighted means ...
fcoverage_naive, fnor_wid_naive, fcoverage_cqr, fnor_wid_cqr, fcoverage_quantile, fnor_wid_quantile, fcoverage_weighted, fnor_wid_weight,fcoverage_weightedCQR, fnor_wid_weightCQR, fcoverage_enbpi, fnor_wid_enbpi = 0,0,0,0,0,0,0,0,0,0,0,0
n = X.station_id.nunique()
N = 0
# actually use station_id !
# coverage_encqr, nor_wid_encqr,
for stat in tqdm(X.station_id.unique()):
    coverage_naive, nor_wid_naive, coverage_cqr, nor_wid_cqr, coverage_quantile, nor_wid_quantile, coverage_weighted, nor_wid_weight,coverage_weightedCQR, nor_wid_weightCQR, coverage_enbpi, nor_wid_enbpi, n_test = result_conformal(stat)
    fcoverage_naive += coverage_naive*n_test
    fnor_wid_naive += nor_wid_naive*n_test
    fcoverage_cqr += coverage_cqr*n_test
    fnor_wid_cqr += nor_wid_cqr*n_test
    fcoverage_quantile += coverage_quantile*n_test
    fnor_wid_quantile += nor_wid_quantile*n_test
    fcoverage_weighted += coverage_weighted*n_test
    fnor_wid_weight += nor_wid_weight*n_test
    fcoverage_weightedCQR += coverage_weightedCQR*n_test
    fnor_wid_weightCQR += nor_wid_weightCQR*n_test
    fcoverage_enbpi += coverage_enbpi*n_test
    fnor_wid_enbpi += nor_wid_enbpi*n_test
    N += n_test

100%|██████████| 102/102 [1:04:57<00:00, 38.21s/it]


In [ ]:
print("coverage naive ", fcoverage_naive/N, " normalized width ",fnor_wid_naive/N)
print("coverage CQR ", fcoverage_cqr/N, " normalized width ",fnor_wid_cqr/N)
print("coverage quantile ", fcoverage_quantile/N, " normalized width ",fnor_wid_quantile/N)
print("coverage weighetd naive ", fcoverage_weighted/N, " normalized width ",fnor_wid_weight/N)
print("coverage weighted CQR ", fcoverage_weightedCQR/N, " normalized width ",fnor_wid_weightCQR/N)
print("coverage enbip ", fcoverage_enbpi/N, " normalized width ",fnor_wid_enbpi/N)

coverage naive  0.8755155032839469  normalized width  0.7238463102717966
coverage CQR  0.881625171834428  normalized width  0.7681463695042399
coverage quantile  0.8843745226821444  normalized width  0.804393993288511
coverage weighetd naive  0.8929280586528181  normalized width  0.7858998613290046
coverage weighted CQR  0.8524515045058806  normalized width  0.804393993288511
coverage enbip  0.6592332365969146  normalized width  0.35872067985208766


In [ ]:
# conformalize each station
# NAIVE/CQR/QUANTILE/WeightNAIVE/WeightCQR/EnbPI
def plot_conformal(stat_id):
    rho = 0.99
    # get station data
    X_train_stat = X_train.loc[X_train.station_id == stat_id,:]
    Y_train_stat = Y_train.loc[X_train.station_id == stat_id, "OAtot_PMF"]
    X_cal_stat = X_cal.loc[X_cal.station_id == stat_id, covars]
    X_test_stat = X_test.loc[X_test.station_id == stat_id, covars]
    Y_cal_stat = Y_cal.loc[Y_cal.station_id == stat_id , "OAtot_PMF"]
    Y_test_stat = Y_test.loc[Y_test.station_id == stat_id , "OAtot_PMF"]

    # NAIVE
    mapie = MapieRegressor(boost, cv = "prefit", method = "base")
    mapie.fit(X_cal_stat, Y_cal_stat)
    y_pred_naive, y_pis_naive = mapie.predict(X_test_stat, alpha = alpha)
    coverage_naive = regression_coverage_score(Y_test_stat, y_pis_naive[:, 0, 0], y_pis_naive[:, 1, 0])
    wid_naive = (y_pis_naive[:, 1, 0] - y_pis_naive[:, 0, 0]).mean()
    nor_wid_naive = wid_naive/(Y_test_stat.max() - Y_test_stat.min())

    # CQR
    mapie = MapieQuantileRegressor(boost_median, cv = "split", method = "quantile", alpha = alpha)
    mapie.fit(X_train[covars], Y_train.OAtot_PMF, X_calib = X_cal_stat, y_calib = Y_cal_stat)
    y_pred_cqr, y_pis_cqr = mapie.predict(X_test_stat, alpha = alpha)
    # try my own implementation
    coverage_cqr = regression_coverage_score(Y_test_stat, y_pis_cqr[:, 0, 0], y_pis_cqr[:, 1, 0])
    wid_cqr = (y_pis_cqr[:, 1, 0] - y_pis_cqr[:, 0, 0]).mean()
    nor_wid_cqr= wid_cqr/(Y_test_stat.max() - Y_test_stat.min())

    # QUANTILE
    y_pred_quantile = boost_median.predict(X_test_stat)
    y_pis_quantile_l = qlo.predict(X_test_stat)
    y_pis_quantile_u = qup.predict(X_test_stat)
    coverage_quantile = regression_coverage_score(Y_test_stat, y_pis_quantile_l, y_pis_quantile_u)
    wid_quantile = (y_pis_quantile_u - y_pis_quantile_l).mean()
    nor_wid_quantile= wid_quantile/(Y_test_stat.max() - Y_test_stat.min())

    # weighted NAIVE
    y_pred_weighted, y_pis_lower, y_pis_upper = weighted_conformal(boost,  rho, alpha, X_train_stat, Y_train_stat, X_cal_stat, Y_cal_stat, X_test_stat, Y_test_stat)
    coverage_weighted = regression_coverage_score(Y_test_stat, y_pis_lower, y_pis_upper)
    wid_weighted = (y_pis_upper - y_pis_lower).mean()
    nor_wid_weight= wid_weighted/(Y_test_stat.max() - Y_test_stat.min())

    # Weighted CQR
    y_pis_lowerCQR, y_pis_upperCQR = weighted_QCR(clf, clf2, rho , alpha, X_train_stat, Y_train_stat, X_cal_stat, Y_cal_stat, X_test_stat, Y_test_stat)
    coverage_weightedCQR = regression_coverage_score(Y_test_stat, y_pis_lowerCQR, y_pis_upperCQR)
    wid_weightedCQR = (y_pis_upperCQR - y_pis_lowerCQR).mean()
    nor_wid_weightCQR = wid_weightedCQR/(Y_test_stat.max() - Y_test_stat.min())

    # EnbPI
    # get only station id data here
    # get only station id data here
    rf = RandomForestRegressor(random_state=10)
    # n_resamplings = B, length = s, agg_function = maybe better quantile?
    cv_mapiets = BlockBootstrap(n_resamplings=50, length= int(len(Y_train_stat)/2), overlapping=True, random_state=59)
    # fit EnbPI
    mapie_enbpi = MapieTimeSeriesRegressor(rf, method = "enbpi",
                                            cv = cv_mapiets, agg_function = "mean", n_jobs = -1)
    mapie_enbpi = mapie_enbpi.fit(pd.concat((X_train_stat[covars], X_cal_stat[covars]),axis = 0), pd.concat((Y_train_stat, Y_cal_stat),axis = 0))
    y_pred_enbpi, y_pis_enbpi = mapie_enbpi.predict(X_test_stat, alpha = alpha, ensemble = True, optimize_beta = False) # # maybe betteer false #true or false
    coverage_enbpi = regression_coverage_score(Y_test_stat, y_pis_enbpi[:, 0, 0], y_pis_enbpi[:, 1, 0])
    wid_enbpi = (y_pis_enbpi[:, 1, 0] - y_pis_enbpi[:, 0, 0]).mean()
    nor_wid_enbpi= wid_enbpi/(Y_test_stat.max() - Y_test_stat.min())

    n_test = len(Y_test_stat)

    return y_pred_naive, y_pis_naive, y_pred_cqr, y_pis_cqr, y_pis_quantile_l, y_pis_quantile_u, y_pis_lower, y_pis_upper, y_pis_lowerCQR, y_pis_upperCQR, y_pred_enbpi, y_pis_enbpi

In [ ]:
# get values to plot
y_pred = {}
y_pis_naive = {}
y_pred_cqr = {}
y_pis_cqr = {}
y_pis_quantile_l = {}
y_pis_quantile_u = {}
y_pis_lower = {}
y_pis_upper = {}
y_pis_lowerCQR = {}
y_pis_upperCQR = {}
y_pred_enbpi = {}
y_pis_enbpi = {}

for stat in tqdm(X.station_id.unique()):
    y_pred[stat], y_pis_naive[stat],y_pred_cqr[stat], y_pis_cqr[stat], y_pis_quantile_l[stat],y_pis_quantile_u[stat], y_pis_lower[stat],y_pis_upper[stat],y_pis_lowerCQR[stat], y_pis_upperCQR[stat], y_pred_enbpi[stat], y_pis_enbpi[stat]  = plot_conformal(stat)

100%|██████████| 102/102 [54:11<00:00, 31.88s/it]


In [ ]:
# plot for individual station the comparison
def plot_comparison(station_name):
    #sns.set_style("whitegrid")
    station_id = data.loc[data.station == station_name, "station_id"].iloc[0]
    y_obs = Y_test.loc[Y_test.station_id == station_id,"OAtot_PMF"].reset_index(drop=True)
    y_camx = (X_test.loc[Y_test.station_id == station_id,["HOA_CAMX","BBOA_CAMX","OOAtot_CAMX"]].apply(np.sum, axis = 1)).reset_index(drop=True)
    fig, ax = plt.subplots(3,2, figsize = (25,15))
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor = "black")
    y_loc = np.max( (y_obs.max(), y_camx.max()) )*0.75
    # NAIVE
    ax[0,0].plot(y_pred[station_id], label = "Prediction", color = "green")
    ax[0,0].plot(y_obs, label ="Observed OA", color = "blue")
    ax[0,0].plot(y_camx, label ="CAMx", color = "orange")
    ax[0,0].fill_between(y_obs.index, y_pis_naive[station_id][:,0,0], y_pis_naive[station_id][:,1,0], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[0,0].set_title("Standard CP", fontsize = 20)
    ax[0,0].set_ylabel("OA", fontsize = 20)
    ax[0,0].set_yticklabels(ax[0,0].get_yticks(),fontsize = 16)
    ax[0,0].set_xticklabels(ax[0,0].get_xticks(),fontsize = 16)
    naive_coverage = np.round(regression_coverage_score(y_obs, y_pis_naive[station_id][:,0,0], y_pis_naive[station_id][:,1,0]),2)
    naive_width = (y_pis_naive[station_id][:,1,0] - y_pis_naive[station_id][:,0,0]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (naive_coverage, ),
        r"$Width=%.2f$" % (naive_width, )))
    ax[0,0].text(1,y_loc, textstr, bbox = props,fontsize = 20)
    # CQR
    ax[0,1].plot(y_pred[station_id], label = "Prediction", color = "green")
    ax[0,1].plot(y_obs, label ="Observed OA", color = "blue")
    ax[0,1].plot(y_camx, label ="CAMx", color = "orange")
    ax[0,1].fill_between(y_obs.index, y_pis_cqr[station_id][:,0,0], y_pis_cqr[station_id][:,1,0], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[0,1].set_title("CQR", fontsize = 20)
    cqr_coverage = np.round(regression_coverage_score(y_obs, y_pis_cqr[station_id][:,0,0], y_pis_cqr[station_id][:,1,0]),2)
    cqr_width = (y_pis_cqr[station_id][:,1,0] - y_pis_cqr[station_id][:,0,0]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (cqr_coverage, ),
        r"$Width=%.2f$" % (cqr_width, )))
    ax[0,1].text(1,y_loc, textstr, bbox = props, fontsize = 20)
    # QUANTILE (ses same prediction as cqr)
    ax[1,0].plot(y_pred[station_id], label = "Prediction", color = "green")
    ax[1,0].plot(y_obs, label ="Observed OA", color = "blue")
    ax[1,0].plot(y_camx, label ="CAMx", color = "orange")
    ax[1,0].fill_between(y_obs.index, y_pis_quantile_l[station_id], y_pis_quantile_u[station_id], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[1,0].set_title("Quantile regression", fontsize = 20)
    ax[1,0].set_ylabel("OA", fontsize = 20)
    ax[1,0].set_yticklabels(ax[1,0].get_yticks(),fontsize = 16)
    ax[1,0].set_xticklabels(ax[1,0].get_xticks(),fontsize = 16)
    quantile_coverage = np.round(regression_coverage_score(y_obs, y_pis_quantile_l[station_id], y_pis_quantile_u[station_id]),2)
    quantile_width = (y_pis_quantile_u[station_id] - y_pis_quantile_l[station_id]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (quantile_coverage, ),
        r"$Width=%.2f$" % (quantile_width, )))
    ax[1,0].text(1,y_loc, textstr, bbox = props, fontsize = 20)
    # WEIGHETD NAIVE (same prediction as naive)
    ax[1,1].plot(y_pred[station_id], label = "Prediction",color = "green")
    ax[1,1].plot(y_obs, label ="Observed OA",color = "blue")
    ax[1,1].plot(y_camx, label ="CAMx", color = "orange")
    ax[1,1].fill_between(y_obs.index, y_pis_lower[station_id], y_pis_upper[station_id], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[1,1].set_title("Weighted Standard CP", fontsize = 20)
    wnaive_coverage = np.round(regression_coverage_score(y_obs, y_pis_lower[station_id], y_pis_upper[station_id]),2)
    wnaive_width = (y_pis_upper[station_id] - y_pis_lower[station_id]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (wnaive_coverage, ),
        r"$Width=%.2f$" % (wnaive_width)))
    ax[1,1].text(1,y_loc, textstr, bbox = props, fontsize = 20)
    # WEIGHTED CQR (same prediction as cqr)
    ax[2,0].plot(y_pred[station_id], label = "Prediction",color ="green")
    ax[2,0].plot(y_obs, label ="Observed OA", color = "blue")
    ax[2,0].plot(y_camx, label ="CAMx",color = "orange")
    ax[2,0].fill_between(y_obs.index, y_pis_lowerCQR[station_id], y_pis_upperCQR[station_id], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[2,0].set_title("Weighted CQR", fontsize = 20)
    ax[2,0].set_ylabel("OA", fontsize = 20)
    ax[2,0].set_xticklabels(ax[2,0].get_xticks(),fontsize = 16)
    ax[2,0].set_yticklabels(ax[2,0].get_yticks(),fontsize = 16)
    wcqr_coverage = np.round(regression_coverage_score(y_obs, y_pis_lowerCQR[station_id], y_pis_upperCQR[station_id]),2)
    wcqr_width = (y_pis_upperCQR[station_id] - y_pis_lowerCQR[station_id]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (wcqr_coverage, ),
        r"$Width=%.2f$" % (wcqr_width, )))
    ax[2,0].text(1,y_loc, textstr, bbox = props, fontsize = 20)
    # ENBPI
    ax[2,1].plot(y_pred_enbpi[station_id], label = "Prediction", color = "green")
    ax[2,1].plot(y_obs, label ="Observed OA",color = "blue")
    ax[2,1].plot(y_camx, label ="CAMx",color = "orange")
    ax[2,1].fill_between(y_obs.index, y_pis_enbpi[station_id][:,0,0], y_pis_enbpi[station_id][:,1,0], color = "green", alpha = 0.2, label = "Prediction Interval")
    ax[2,1].set_title("EnbPI", fontsize = 20)
    ax[2,1].set_yticklabels(ax[2,1].get_yticks(),fontsize = 16)
    ax[2,1].set_xticklabels(ax[2,1].get_xticks(),fontsize = 16)
    enbpi_coverage = np.round(regression_coverage_score(y_obs, y_pis_enbpi[station_id][:,0,0], y_pis_enbpi[station_id][:,1,0]),2)
    enbpi_width = (y_pis_enbpi[station_id][:,1,0] - y_pis_enbpi[station_id][:,0,0]).mean()
    textstr = "\n".join((
        r"$Coverage=%.2f$" % (enbpi_coverage, ),
        r"$Width=%.2f$" % (enbpi_width, )))
    ax[2,1].text(1,y_loc, textstr, bbox = props, fontsize = 20)
    plt.legend(loc='upper center', bbox_to_anchor=(-0.1, -0.15),
                    fancybox=True, shadow=True, ncol=5, fontsize = 20)


    fig.suptitle(station_id + ": " + station_name, fontsize = 30)
    for ax in ax.flat:
        ax.label_outer()

In [ ]:
interact(plot_comparison, station_name = data.station.unique())

In [ ]:
from matplotlib.offsetbox import AnnotationBbox, TextArea
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import randint, uniform
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from mapie.metrics import (regression_coverage_score,
                           regression_mean_width_score)
from mapie.subsample import Subsample
random_state = 23
rng = np.random.default_rng(random_state)
round_to = 3
warnings.filterwarnings("ignore")

In [ ]:
# plot all methods x vs y randomly getting stations
def sort_y_values(y_test, y_pred, y_pis):
    """
    Sorting the dataset in order to make plots using the fill_between function.
    """
    indices = np.argsort(y_test)
    y_test_sorted = np.array(y_test)[indices]
    y_pred_sorted = y_pred[indices]
    y_lower_bound = y_pis[:, 0, 0][indices]
    y_upper_bound = y_pis[:, 1, 0][indices]
    return y_test_sorted, y_pred_sorted, y_lower_bound, y_upper_bound

def sort_y_values2(y_test, y_pred, y_lower, y_upper):
    """
    Sorting the dataset in order to make plots using the fill_between function.
    """
    indices = np.argsort(y_test)
    y_test_sorted = np.array(y_test)[indices]
    y_pred_sorted = y_pred[indices]
    y_lower_bound = y_lower[indices]
    y_upper_bound = y_upper[indices]
    return y_test_sorted, y_pred_sorted, y_lower_bound, y_upper_bound


def plot_prediction_intervals(
    title,
    axs,
    y_test_sorted,
    y_pred_sorted,
    lower_bound,
    upper_bound,
    coverage,
    width,
    num_plots_idx
):
    """
    Plot of the prediction intervals for each different conformal
    method.
    """

    axs.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    axs.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))

    lower_bound_ = np.take(lower_bound, num_plots_idx)
    y_pred_sorted_ = np.take(y_pred_sorted, num_plots_idx)
    y_test_sorted_ = np.take(y_test_sorted, num_plots_idx)

    error = np.abs(y_pred_sorted_-lower_bound_)

    warning1 = y_test_sorted_ > y_pred_sorted_+error
    warning2 = y_test_sorted_ < y_pred_sorted_-error
    warnings = warning1 + warning2
    axs.errorbar(
        y_test_sorted_[~warnings],
        y_pred_sorted_[~warnings],
        yerr=error[~warnings],
        capsize=5, marker="o", elinewidth=2, linewidth=0, color = "blue",
        label="Inside prediction interval"
        )
    axs.errorbar(
        y_test_sorted_[warnings],
        y_pred_sorted_[warnings],
        yerr=error[warnings],
        capsize=5, marker="o", elinewidth=2, linewidth=0, color="red",
        label="Outside prediction interval"
        )
    axs.scatter(
        y_test_sorted_[warnings],
        y_test_sorted_[warnings],
        marker="*", color="green",
        label="True value"
    )
    axs.set_xlabel("Observed OA")
    axs.set_ylabel("Predicted OA")
    #ab = AnnotationBbox(TextArea(
           # f"Coverage: {np.round(coverage, round_to)}\n"
          #  + f"Interval width: {np.round(width, round_to)}"),xy=(np.min(y_test_sorted_)*3, np.max(y_pred_sorted_+error)*0.95),)

    lims = [
        np.min([axs.get_xlim(), axs.get_ylim()]),  # min of both axes
        np.max([axs.get_xlim(), axs.get_ylim()]),  # max of both axes
    ]
    pt = (0, 0)
    axs.axline(pt, slope=1, color='black', label = "x=y")
    #axs.plot(0,1, '--', alpha=0.75, color="black", label="x=y")
    #axs.add_artist(ab)
    axs.set_title(title, fontweight='bold', fontsize = 20)
    axs.set_ylabel(axs.get_ylabel(), fontsize = 18)
    axs.set_xlabel(axs.get_xlabel(), fontsize = 18)
    # change the fontsize
    axs.tick_params(axis='x', labelsize=16)
    axs.tick_params(axis='y', labelsize=16)
    #fig.supxlabel('common_x')
    #fig.supylabel('common_y')

In [ ]:
# multiple stations plot
fig, ax = plt.subplots(3,2, figsize = (20,12), sharey = True)
sns.set_style("whitegrid")
axs = [ ax[0,0], ax[0,1], ax[1,0], ax[1,1], ax[2,0], ax[2,1]]
strategy = ["Standard CP", "CQR", "Quantile Regression", "Weighted CP", "Weighted CQR", "EnbPI"]
perc_obs_plot = 0.05
# plot only they have at least 10 data points in the test set
keep = (Y_test.groupby("station_id")["OAtot_PMF"].size() > 10).reset_index()
Y_test2 = Y_test.set_index("station_id").join(keep.set_index("station_id"), rsuffix = "pr")
Y_test2 = Y_test2.loc[Y_test2.OAtot_PMFpr == True,:]
for idx, stat_id in enumerate(Y_test2.reset_index().station_id.unique()):
    y_test = Y_test.loc[Y_test.station_id == stat_id, "OAtot_PMF"].reset_index(drop=True)
    num_plots = rng.choice(len(y_test), int(perc_obs_plot*len(y_test)), replace=False)
    y_test_sorted = {}
    y_pred_sorted = {}
    y_lower_sorted = {}
    y_upper_sorted = {}
    y_test_sorted["Standard CP"] , y_pred_sorted["Standard CP"], y_lower_sorted["Standard CP"], y_upper_sorted["Standard CP"] = sort_y_values(y_test, y_pred[stat_id], y_pis_naive[stat_id])
    y_test_sorted["CQR"] , y_pred_sorted["CQR"], y_lower_sorted["CQR"], y_upper_sorted["CQR"]= sort_y_values(y_test, y_pred[stat_id], y_pis_cqr[stat_id])
    y_test_sorted["Quantile Regression"], y_pred_sorted["Quantile Regression"], y_lower_sorted["Quantile Regression"], y_upper_sorted["Quantile Regression"] = sort_y_values2(y_test,y_pred[stat_id], y_pis_quantile_l[stat_id], y_pis_quantile_u[stat_id])
    y_test_sorted["Weighted CP"] , y_pred_sorted["Weighted CP"], y_lower_sorted["Weighted CP"], y_upper_sorted["Weighted CP"] = sort_y_values2(y_test, y_pred[stat_id], y_pis_lower[stat_id], y_pis_upper[stat_id])
    y_test_sorted["Weighted CQR"] , y_pred_sorted["Weighted CQR"], y_lower_sorted["Weighted CQR"], y_upper_sorted["Weighted CQR"] = sort_y_values2(y_test, y_pred[stat_id], y_pis_lowerCQR[stat_id], y_pis_upperCQR[stat_id])
    y_test_sorted["EnbPI"] , y_pred_sorted["EnbPI"], y_lower_sorted["EnbPI"], y_upper_sorted["EnbPI"] = sort_y_values(y_test, y_pred_enbpi[stat_id], y_pis_enbpi[stat_id])


    for idx, (axis, strat) in enumerate(zip(axs, strategy)):
        plot_prediction_intervals(
                strat,
                axis,
                y_test_sorted[strat],
                y_pred_sorted[strat],
                y_lower_sorted[strat],
                y_upper_sorted[strat],
                10,
                2,
                10 # num_plots
                )



lines_labels = [axis.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(_, []) for _ in zip(*lines_labels)]
plt.legend(
                      lines[:2] + lines[-2:], labels[:2] + labels[-2:],
                      loc='upper center',
                      bbox_to_anchor=(0, -0.225),
                      fancybox=True,
                      shadow=True,
                      ncol=2, fontsize = 20
                  )
for ax in ax.flat:
        ax.label_outer()
plt.show()

In [ ]:
 def plot_stat(station_name):
    stat_id = data.loc[data.station == station_name, "station_id"].iloc[0]
    y_test = Y_test.loc[Y_test.station_id == stat_id, "OAtot_PMF"].reset_index(drop=True)
    y_test_sorted = {}
    y_pred_sorted = {}
    y_lower_sorted = {}
    y_upper_sorted = {}
    y_test_sorted["Standard CP"] , y_pred_sorted["Standard CP"], y_lower_sorted["Standard CP"], y_upper_sorted["Standard CP"] = sort_y_values(y_test, y_pred[stat_id], y_pis_naive[stat_id])
    y_test_sorted["CQR"] , y_pred_sorted["CQR"], y_lower_sorted["CQR"], y_upper_sorted["CQR"]= sort_y_values(y_test, y_pred[stat_id], y_pis_cqr[stat_id])
    y_test_sorted["Quantile Regression"], y_pred_sorted["Quantile Regression"], y_lower_sorted["Quantile Regression"], y_upper_sorted["Quantile Regression"] = sort_y_values2(y_test,y_pred[stat_id], y_pis_quantile_l[stat_id], y_pis_quantile_u[stat_id])
    y_test_sorted["Weighted CP"] , y_pred_sorted["Weighted CP"], y_lower_sorted["Weighted CP"], y_upper_sorted["Weighted CP"] = sort_y_values2(y_test, y_pred[stat_id], y_pis_lower[stat_id], y_pis_upper[stat_id])
    y_test_sorted["Weighted CQR"] , y_pred_sorted["Weighted CQR"], y_lower_sorted["Weighted CQR"], y_upper_sorted["Weighted CQR"] = sort_y_values2(y_test, y_pred[stat_id], y_pis_lowerCQR[stat_id], y_pis_upperCQR[stat_id])
    y_test_sorted["EnbPI"] , y_pred_sorted["EnbPI"], y_lower_sorted["EnbPI"], y_upper_sorted["EnbPI"] = sort_y_values(y_test, y_pred_enbpi[stat_id], y_pis_enbpi[stat_id])




    fig, ax = plt.subplots(3,2, figsize = (24,12), sharey = True)
    sns.set_style("whitegrid")
    axs = [ ax[0,0], ax[0,1], ax[1,0], ax[1,1], ax[2,0], ax[2,1]]
    strategy = ["Standard CP", "CQR", "Quantile Regression", "Weighted CP", "Weighted CQR","EnbPI"]
    perc_obs_plot = 1
    num_plots = rng.choice(len(y_test), int(perc_obs_plot*len(y_test)), replace=False)
    for axis, strat in zip(axs, strategy):
        plot_prediction_intervals(
                    strat,
                    axis,
                    y_test_sorted[strat],
                    y_pred_sorted[strat],
                    y_lower_sorted[strat],
                    y_upper_sorted[strat],
                    10,
                    2,
                    num_plots
                    )

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(_, []) for _ in zip(*lines_labels)]
    plt.legend(
                lines[:4], labels[:4],
                loc='upper center',
                bbox_to_anchor=(0, -0.225),
                fancybox=True,
                shadow=True,
                ncol=2, fontsize = 20
            )
    for ax in ax.flat:
        ax.label_outer()
    plt.suptitle(station_name, fontsize = 30)
    plt.show()

In [ ]:
interact(plot_stat, station_name = Y_test.station.unique())